In [119]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors
import nltk
import spacy
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from spacy.lang.en import English
 

In [120]:
movies=pd.read_csv('data.csv')


In [121]:
new_movies=movies[['id', 'title', 'genre', 'overview']]
df = pd.DataFrame(new_movies)


In [122]:
#new_movies['tag'] = new_movies['genre'] + new_movies['overview']
#newData=new_movies
df['tag'] = df['genre'].str.cat(df['overview'], sep=' ')


In [123]:
df = df.drop(columns=['genre', 'overview'])
df = df.dropna(subset=['tag'])

abc


In [124]:
# # Download NLTK resources
# nltk.download('punkt')
# nltk.download('wordnet')
# nltk.download('omw-1.4')

# # Initialize SpaCy
# nlp = spacy.load('en_core_web_sm')


# # Check the DataFrame columns
# print("DataFrame columns:", df.columns)

# # NLTK tokenization, stemming, and lemmatization using lambda functions
# lemmatizer = nltk.WordNetLemmatizer()
# stemmer = PorterStemmer()

# nltk_tokenize_stem_and_lemmatize = lambda text: [
#     (token, stemmer.stem(token), lemmatizer.lemmatize(token)) for token in word_tokenize(text)
# ]

# # SpaCy tokenization and lemmatization using lambda functions
# spacy_tokenize_and_lemmatize = lambda text: [
#     (token.text, token.lemma_) for token in nlp(text)
# ]

# # Apply tokenization, stemming, and lemmatization to DataFrame
# df['nltk_processed'] = df['tag'].apply(nltk_tokenize_stem_and_lemmatize)  
# df['spacy_processed'] = df['tag'].apply(spacy_tokenize_and_lemmatize)    

# # # Print results
# # print("Tokenization, Stemming, and Lemmatization using NLTK:")
# # for _, row in df.iterrows():
# #     print(f"Original: {row['tag']}")  
# #     print(f"NLTK Processed: {row['nltk_processed']}\n")

# # print("Tokenization and Lemmatization using SpaCy:")
# # for _, row in df.iterrows():
# #     print(f"Original: {row['tag']}")  
# #     print(f"SpaCy Processed: {row['spacy_processed']}\n")

In [125]:
# Split the data into train and test sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [126]:
vectorizer = TfidfVectorizer()
tfidf_matrix_train = vectorizer.fit_transform(train_df['tag'])
tfidf_matrix_test = vectorizer.transform(test_df['tag'])
# tfidf_matrix_test

In [127]:
# # Initialize the k-NN model
# k = 2  # Number of neighbors to use
# knn_model = NearestNeighbors(n_neighbors=k, metric='cosine')

# # Fit the model on the training data
# knn_model.fit(tfidf_matrix_test)

# # Function to get k-nearest neighbors
# def get_k_nearest_neighbors(model, query_vector):
#     distances, indices = model.kneighbors(query_vector)
#     return indices[0]


In [128]:
def get_knn_recommendations(movie_vector, knn, top_n=5):
    distances, indices = knn.kneighbors(movie_vector, n_neighbors=top_n)
    return indices[0] 



In [132]:
def evaluate_recommendations(test_df, train_df, knn, vectorizer):
    y_true = []
    y_pred = []

    for idx, row in test_df.iterrows():
        if row['title'] in train_df['title'].values:
            movie_index = train_df[train_df['title'] == row['title']].index[0]
            movie_vector = tfidf_matrix_test[idx]
            
            recommended_indices = get_knn_recommendations(movie_vector, knn)
            recommended_titles = train_df.iloc[recommended_indices]['title'].tolist()

            relevant_items = [row['title']]
            recommended_items = [train_df.iloc[i]['title'] for i in recommended_indices]
            
            y_true.extend([1 if item in relevant_items else 0 for item in recommended_items])
            y_pred.extend([1 if item in relevant_items else 0 for item in recommended_items])
    
    precision = precision_score(y_true, y_pred, average='binary')
    recall = recall_score(y_true, y_pred, average='binary')
    return precision, recall

    




abc
